# <span style="color:blue">Modeling atmospheres and their evolution with `exo_k`: a practical guide</span>

*Author: Jeremy Leconte (CNRS/LAB/Univ. Bordeaux)*

Since `exo_k` allows anyone to easily read radiative data and interpolate into them, it seemed
silly not to incorporate a module to test this directly in a real-life situation.
This is why we implemented the `Atm` class that will allow you to build atmospheres with any composition,
and compute their transmission/emission spectra with any combination of radiative data that the library can handle.
It can also compute the radiative heating rates in each atmospheric layer.

But why stop there? Indeed, heating rates allow you to compute the radiative 
equilibrium of an atmosphere. One thing leading to another, we ended up
with a full fledged atmospheric model able to compute the evolution of an atmosphere
in time that accounts for radiation (off course), dry convection, but also moist convection,
condensation, turbulent diffusion, etc. 
This model is encapsulated in the :class:`~exo_k.atm_evolution.atm_evol.Atm_evolution` class that will be presented below. 

## Initialization

First, let's make some common initializations.   
We also import our library: `exo_k`.

For this to work, you need to have installed the library by either typing
```
pip install exo_k
```
anywhere or
```
pip install -e .
```
at the root of exo_k directory (the location where you first saw this tutorial).

In [1]:
import exo_k as xk
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import time,sys,os

xk.Settings().set_mks(True)

We also set up the generic path toward the radiative data

In [2]:
xk.Settings().set_search_path('data/xsec', path_type='xtable')
xk.Settings().set_search_path('data/corrk', path_type='ktable')
xk.Settings().set_search_path('data/cia', path_type='cia')
xk.Settings().search_path

{'ktable': ['/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/corrk'],
 'xtable': ['/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/xsec'],
 'cia': ['/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia'],
 'aerosol': ['/Users/jleconte/atmosphere/RadiativeTransfer/exo_k']}

In [3]:
# Uncomment the line below if you want to enable interactive plots
#%matplotlib notebook
plt.rcParams["figure.figsize"] = (7,4)
from matplotlib import cycler
colors = cycler('color',[plt.cm.inferno(i) for i in np.linspace(0.1,1,5)])
plt.rc('axes', axisbelow=True, grid=True, labelcolor='dimgray', labelweight='bold', prop_cycle=colors)
plt.rc('grid', linestyle='solid')
plt.rc('xtick', direction='in', color='dimgray')
plt.rc('ytick', direction='in', color='dimgray')
plt.rc('lines', linewidth=1.5)

# <span style="color:blue">The `Atm` class: An atmospheric radiative-transfer model</span>

Whether you want to test various assumptions (will my resolution be enough, are my initial data adequate, what weights should I use, etc.) or directly simulate the transmission/emission spectrum of a planet, we implemented a simple 1d atmosphere model so that you can do just that.

This revolves around the `Atm()` class, which defines an atmosphere with Pressure, Temperature, and composition profiles. In essence, we are using the structure of the `Gas_mix` object and add the additional attributes that make an atmosphere more than just some gas with a given pressure, temperature, and composition: a gravity, a planetary radius, etc. 

As `Xtable()` objects can be used in a `Kdatabase()`, you can do the equivalent of Line by Line radiative transfer. 

If you want to start from very high-res spectra, first load them into a `Xtable()`, and then into a Database.

## Loading the radiative data that we will use in our atmospheric model

### Molecular absorptions: The `Kdatabase`

The first thing that we need to do is to create a `Kdatabase` that will contain the data
for all the molecules that we want to include in our atmosphere
and for which we have cross sections
(whether they are monochromatic cross sections of correlated-k coefficients). 

In the example below we will treat a simple examples with only CO2 and H2O.
Other gases, if any, will only contribute through continuum absorptions.

In [4]:
k_db=xk.Kdatabase(['CO2', 'H2O'],'R300_0.3-50mu.ktable.SI')
k_db

The available molecules are: 
CO2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/corrk/CO2_R300_0.3-50mu.ktable.SI.h5
H2O->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/corrk/H2O_R300_0.3-50mu.ktable.SI.h5
All tables share a common spectral grid
All tables share a common logP-T grid
All tables share a common p unit
All tables share a common kdata unit

### Collision induced absorptions

Now we can load CIA data in a `CIA_database`. If you do not want to include such constributions,
do not provide any CIA database when instantiating your `Atm` object or use `cia_database=None`.

In [5]:
cia_db=xk.CIAdatabase(molecules=['N2','H2O','H2','He'], mks=True)
cia_db

The available molecule pairs are: 
N2-N2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/N2-N2_2011.h5
N2-H2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/N2-H2_2011.h5
H2O-N2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2O-N2_continuumTurbet_MT_CKD3.3.h5
H2O-H2O->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2O-H2O_continuumTurbet_MT_CKD3.3.h5
H2O-H2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2O-H2_fromMT.h5
H2-H2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2-H2_2011.h5
H2-He->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2-He_2011.h5
All tables do NOT have common spectral grid
You will need to run sample before using the database

Just like with the `Kdatabase`, all the CIA tables must use the same spectral axis (and the same as the `Kdatabase`)
so we usually need to resample our database on the same grid as follows.

In [6]:
cia_db.sample(k_db.wns)
cia_db

The available molecule pairs are: 
N2-N2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/N2-N2_2011.h5
N2-H2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/N2-H2_2011.h5
H2O-N2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2O-N2_continuumTurbet_MT_CKD3.3.h5
H2O-H2O->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2O-H2O_continuumTurbet_MT_CKD3.3.h5
H2O-H2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2O-H2_fromMT.h5
H2-H2->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2-H2_2011.h5
H2-He->/Users/jleconte/atmosphere/RadiativeTransfer/exo_k/data/cia/H2-He_2011.h5
All tables share a common spectral grid

### Rayleigh scattering

For the moment, the data for Rayligh scattering are hardcoded inside the library. So there is no need
to load them. You'll just have to use the `rayleigh=True` keyword when calling the relevant methods. 

## Building an atmosphere

### Simple troposphere/stratosphere parametrization

There are several ways to build an atmosphere. If you just want a simple atmospheric structure
with an isothermal stratosphere on top of a dry troposphere with a fixed adiabatic gradient,
you can just use the following options

In [ ]:
Nlay = 50 # Number of layers
Tsurf = 220. # K => Surface temperature
Tstrat = 100.  # K => Temperature of the stratosphere
psurf = 640. # Pa => Surface pressure
ptop = 1.e-1 # Pa => Pressure at the model top
grav = 3.4 # m/s^2 => Surface gravity
rcp = 0.28 # dimless => R/cp
Rp = None # m => Planetary radius

composition = {'CO2':'background', 'H2O':1.e-6}
# Volume molar concentrations of the various species
# One gas can be set to `background`.
# It's volume mixing ratio will be automatically computed to fill the atmosphere.

atm_mars = xk.Atm(psurf=psurf, ptop=ptop,
             Tsurf=Tsurf, Tstrat=Tstrat,
             grav=grav, rcp = rcp, Rp=Rp,
             composition = composition, Nlay=Nlay,
             k_database = k_db, cia_database = cia_db)

In [ ]:
fig,axs=plt.subplots(1,2,sharey=False,figsize=(7,4))  
atm_mars.plot_profile(axs[0], yscale='log')
atm_mars.plot_profile(axs[1], use_altitudes = True)
fig.tight_layout()

If the stratospheric temperature is set to `None`, the profile will be isothermal at T=Tsurf.

In [ ]:
Nlay = 100 # Number of layers
Tsurf = 1000. # K => Surface temperature
Tstrat = None  # K => Temperature of the stratosphere
psurf = 1.e6 # Pa => Surface pressure
ptop = 1.e-4 # Pa => Pressure at the model top
grav = 10. # m/s^2 => Surface gravity
rcp = 0.28 # dimless => R/cp 
Rp = 1.*u.Rjup # m => Planetary radius

composition = {'H2':'background', 'He':0.09, 'H2O':1.e-3}

atm_hotjup = xk.Atm(psurf=psurf, ptop=ptop,
             Tsurf=Tsurf, Tstrat=Tstrat,
             grav=grav, rcp = rcp, Rp=Rp,
             composition = composition, Nlay=Nlay,
             k_database = k_db, cia_database = cia_db)

fig,axs=plt.subplots(1,2,sharey=False,figsize=(7,4))  
atm_hotjup.plot_profile(axs[0], yscale='log')
atm_hotjup.plot_profile(axs[1], use_altitudes = True)
fig.tight_layout()

### Using your own profiles

If you want, you can completely control the profiles you put in your `Atm`. For this, you just need to provide arrays for `logplay`, `tlay`, and the various concentrations.

The atmosphere is defined from the top down so the first value is the top of atmosphere and the last is the surface.

Here is an example for an Earth with a moist troposphere near saturation

In [ ]:
Nlay = 100 # Number of layers
Tsurf = 300. # K => Surface temperature
Tstrat = 200.  # K => Temperature of the stratosphere
psurf = 1.e5 # Pa => Surface pressure
ptop = 1.e-1 # Pa => Pressure at the model top
grav = 9.81 # m/s^2 => Surface gravity
rcp = 0.28 # dimless => R/cp #create the layer presure grid
Rp = 1.*u.Rearth # m => Planetary radius

logplay=np.linspace(np.log10(ptop),np.log10(psurf), Nlay)
# design your adiabat
tlay=Tsurf*(10**logplay/psurf)**rcp
tlay=np.where(tlay<Tstrat,Tstrat,tlay)

# create your composition. Here, only water is variable and follows saturation.
h2o_thermo_params={'Latent_heat_vaporization': 2.486e6, 'cp_vap': 4.228e3, 'Mvap': 18.e-3,
            'T_ref': 273., 'Psat_ref': 612.}
h2o=xk.Condensing_species(**h2o_thermo_params)
psat_h2o=h2o.Psat(tlay)
x_h2o = psat_h2o / (psat_h2o + 10**logplay)

#Loads the atmosphere and computes some intermediate things
atm_earth=xk.Atm(logplay = logplay, tlay = tlay, grav = grav,
                Rp = Rp, rcp = rcp, 
                composition = {'CO2':4.e-4,'H2O':x_h2o,'N2':'background'},
                k_database = k_db, cia_database = cia_db)

In [ ]:
fig,axs=plt.subplots(1,2,sharey=True,figsize=(7,4))  
atm_earth.plot_profile(axs[0], yscale='log')
axs[1].plot(x_h2o, atm_earth.play)
axs[1].set_xscale('log')
axs[1].set_xlabel('H2O molar concentration')
fig.tight_layout()

### Changing attributes of an atmosphere

In an atmosphere, many attributes are inter related and cannot be changed separately.
Changing the pressures, for example, will change the mass of the atmosphere, the altitudes, etc.

So, if you want to change some attributes of the instance you are working with after it has been created, it is recommended to use the methods made for that. Here is a non-exhaustive list:
`set_logPT_profile`, `set_T_profile()`, `set_grav()`,
`set_gas()`, `set_Mgas()`, `set_rcp()`, `set_Rp()`, `set_Rstar()`,
`set_k_database()`, `set_cia_database()`, `set_spectral_range()`,
`set_incoming_stellar_flux()`, `set_internal_flux()`.

See the API reference for completeness.

### Optional attributes

At the initialisation, a `Mgas` can be specified to force a molar mass whatever the composition. 

With `wn_range` or `wl_range` one can limit the wavenumber/wavelength range of future computations.

`flux_top_dw` (w/m^2) allows you to specify the stellar flux impinging on the top of atmosphere.
If you do not specify anything else, the spectral energy distribution will be the one of a
blackbody at T=5778K. This temperature can be changed by using the `Tstar` keyword
at the same time.
Alternatively, one can provide a `Spectrum` object with the keyword `stellar_spectrum`.
This spectrum must be in units of power per wavenumber. 
The actual units are irrelevant as this spectrum will be renormalized with the value of
`flux_top_dw` provided. 

## Transmission spectroscopy

To compute the transmission spectrum for your atmosphere, you can use the following method. Among the available options, there is `rayleigh`, `Rstar`, 

In [ ]:
Rs=1.*u.Rsun # m => Radius of the star to compute the transit depth

spec=atm_hotjup.transmission_spectrum(Rstar=Rs, rayleigh=True)

print("Forward model duration:")
%timeit -n 2 -r 5 atm_hotjup.transmission_spectrum(Rstar=Rs, rayleigh=True)

In [ ]:
fig,ax=plt.subplots(1,1,sharex=True,sharey=True)  
spec.plot_spectrum(ax, xscale='log', label='Native corr-k resolution')
spec.bin_down_cp(xk.wavenumber_grid_R(400.,30000.,200)).plot_spectrum(ax,xscale='log',label=r'$R=200$')
spec.bin_down_cp(xk.wavenumber_grid_R(400.,30000.,30)).plot_spectrum(ax,xscale='log',label=r'$R=30$')
ax.set_ylabel('Depth')
ax.legend()
fig.tight_layout()

## Emission spectroscopy

`Exo_k` enables several modes to compute the emission of a planet:

  * `Atm.emission_spectrum(mu0=, **kwargs)`: Simple computation
    using Schwarzschild's equation with mu0 as the cosine of the
    observer zenith angle. This mode cannot handle scattering. The output
    is given as a flux assuming a constant hemispheric intensity, but
    it can be used to compute the specific intensity at various angles by changin mu0. 
  * `Atm.emission_spectrum(mu_quad_order=, **kwargs)`: This mode computes the
    specific intensity at `mu_quad_order` angles and integrates it to yield the
    flux. Exact in the non-scattering limit. Cannot handle scattering.
  * `Atm.emission_spectrum_2stream(**kwargs)`: This mode computes the
    upward and downward fluxes using the constant hemispheric mean two-stream
    approximation of Toon et al. (1989). This mode can handle scattering and provides
    fluxes at all the layer interfaces (levels),
    so that radiative heating rates can be computed. In this case, a stellar flux
    must be specified.
    
As expected, in a non-scattering medium and with a zero surface albedo,
`Atm.emission_spectrum` and `Atm.emission_spectrum_2stream`
provide the same top of atmosphere outgoing flux almost down to machine precision.
The second will be longer as it computes downward fluxes as well.
    

## Emission

In [ ]:
FluxTop=atm_mars.emission_spectrum_2stream(integral=True, wl_range=[0.1,150.0])
SurfFlux=atm_mars.surf_bb(integral=True)
TopBB=atm_mars.top_bb(integral=True)
print("Forward model duration:")
%timeit -n 2 -r 2 atm_mars.emission_spectrum(integral=True, wl_range=[0.1,150.0])
%timeit -n 2 -r 2 atm_mars.emission_spectrum_2stream(integral=True, wl_range=[0.1,150.0])

The `integral=True` keyword specifies that the source function is integrated inside each bin. If `False`, the planck function is just evaluated at the center of the bin. It is faster, and ok for thin bins, but inacurate for large ones.

The `wl_range` range keyword allows you to limit the wavelength range onto wich you want the calculation to be done to save time. 

In [ ]:
print(FluxTop.total,'W/m^2 (BB at tsurf over the computed nu range is ',SurfFlux.total,'W/m^2)')
print('(sigma*T**4 is ',xk.SIG_SB*Tsurf**4,'W/m^2)')

In [ ]:
fig,axs=plt.subplots(1,2,sharey=False,figsize=(8,4))  
SurfFlux.plot_spectrum(axs[0],label='Blackbody ($T_{surf}$)')
TopBB.plot_spectrum(axs[0],label='Blackbody ($T_{strat}$)')
FluxTop.plot_spectrum(axs[0],label='OLR')
SurfFlux.plot_spectrum(axs[1],per_wavenumber=False,label='Blackbody ($T_{surf}$)')
TopBB.plot_spectrum(axs[1],per_wavenumber=False,label='Blackbody ($T_{strat}$)')
FluxTop.plot_spectrum(axs[1],per_wavenumber=False,label='OLR')

axs[0].set_ylabel(r'Flux ($W/m^2/cm^{-1}$)')
axs[1].set_ylabel(r'Flux ($W/m^2/\mu m$)')
for ax in axs:
    ax.legend()
fig.tight_layout()

You can also bin it down, and when spectra have the same wavenumber grid, perform algeabric operations)

In [ ]:
R=100
wavenumber_grid=xk.wavenumber_grid_R(200.,10000.,R)
binned_spec=FluxTop.bin_down_cp(wavenumber_grid)

fig,axs=plt.subplots(1,2,sharey=True)  
FluxTop.plot_spectrum(axs[0],label='OLR')
binned_spec.plot_spectrum(axs[0],label='binned OLR')
(SurfFlux-FluxTop).plot_spectrum(axs[1],label='Surf Blackbody - OLR')

for ax in axs:
    ax.set_ylabel('Flux ($W/m^2/cm^{-1}$)')
    ax.legend()
fig.tight_layout()

In [ ]:
origin = 'upper'
delta = 0.025

x =k_db.wls
y =atm_mars.play[1:]
X, Y = np.meshgrid(x, y)
Z = atm_mars.exp_minus_tau()

nr, nc = Z.shape

plt.rc('axes',grid=False)
fig, ax = plt.subplots(constrained_layout=True)
ax.invert_yaxis()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(left=0.3,right=20.)
CS = ax.contourf(X, Y, Z, 10, cmap=plt.cm.bone, origin=origin)
ax.set_title('Atm. Transmittance')
ax.set_xlabel('Wavelength (mu)')
ax.set_ylabel('Log(Pressure/Pa)')
# Make a colorbar for the ContourSet returned by the contourf call.
cbar = fig.colorbar(CS)
cbar.ax.set_ylabel('Exp(-tau)')
plt.rc('axes',grid=True)
# Add the contour line levels to the colorbar

### Effect of CIAs

In [ ]:
# Computes emission flux for the two cases
FluxTop_cont=atm_earth.emission_spectrum(integral=True,wl_range=[0.1,40.0], mu_quad_order=3)

atm_earth.set_cia_database(None) # remove the CIA
FluxTop=atm_earth.emission_spectrum(integral=True,wl_range=[0.1,40.0], mu_quad_order=3)
atm_earth.set_cia_database(cia_db) # reset the CIA


# Computes surface BlackBody
SurfFlux=atm_earth.surf_bb(integral=True)

# Computes top of atmosphere BlackBody
TopBB=atm_earth.top_bb(integral=True)
print(FluxTop.total,'W/m^2 (BB at tsurf over the computed nu range is ',SurfFlux.total,'W/m^2)')
print('(sigma*T**4 is ',xk.SIG_SB*atm_earth.tlay[-1]**4,'W/m^2)')

In [ ]:
fig,axs=plt.subplots(1,2,sharey=False, figsize=(8,4))  
SurfFlux.plot_spectrum(axs[0],label=r'Blackbody ($T_{surf}$)')
TopBB.plot_spectrum(axs[0],label=r'Blackbody ($T_{strat}$)')
FluxTop.plot_spectrum(axs[0],label=r'OLR w/o CIA')
FluxTop_cont.plot_spectrum(axs[0],label=r'OLR w/ CIA')
SurfFlux.plot_spectrum(axs[1],per_wavenumber=False,label=r'Blackbody ($T_{surf}$)')
TopBB.plot_spectrum(axs[1],per_wavenumber=False,label=r'Blackbody ($T_{strat}$)')
FluxTop.plot_spectrum(axs[1],per_wavenumber=False,label=r'OLR w/o CIA')
FluxTop_cont.plot_spectrum(axs[1],per_wavenumber=False,label=r'OLR w/ CIA')

axs[0].set_ylabel(r'Flux ($W/m^2/cm^{-1}$)')
axs[1].set_ylabel(r'Flux ($W/m^2/\mu m$)')
for ax in axs:
    ax.legend()
#axs[0].invert_yaxis()
fig.tight_layout()

## Heating rates

In [ ]:
T0=300.
Tstrat=180.
Tstar=5770.
database=xk.Kdatabase(['CO2','H2O'],'_R300_0.3-50mu.ktable.SI')
cia_data=xk.CIAdatabase(molecules=['H2','He'], mks=True)
cia_data.sample(database.wns,remove_zeros=True)

atm_ck=xk.Atm(psurf=1.e5,ptop=1.e1,Tsurf=T0,Tstrat=Tstrat,grav=10.,
                    composition={'H2':'background', 'H2O':0.01},
                    rcp=0.2183,Rp=1.*u.Rearth,
                    Nlay=30, Tstar=Tstar, flux_top_dw=None,
                    k_database=database,cia_database=cia_data)
Heat_Rate, Net_flux = atm_ck.heating_rate(flux_top_dw=260., Tstar=Tstar, source=False)
#atm_ck.set_gas({'H2':'background', 'H2O':0.01})
Heat_Rate2, Net_flux2 = atm_ck.heating_rate(flux_top_dw=0.)
Heat_Rate3, Net_flux3 = atm_ck.heating_rate(flux_top_dw=260., Tstar=Tstar,
                                    rayleigh=True, source=False)


In [ ]:
fig,axs=plt.subplots(1,3,sharey=True)
axs[0].set_yscale('log')
axs[0].set_xscale('log')
axs[0].plot(np.abs(Heat_Rate),atm_ck.play, label='Solar Heating')
axs[0].plot(np.abs(Heat_Rate2),atm_ck.play, label='-IR cooling')
axs[0].plot(np.abs(Heat_Rate3),atm_ck.play, label='w rayleigh')
axs[1].plot(Net_flux,atm_ck.play, label='Net solar flux')
axs[1].plot(Net_flux2,atm_ck.play, label='Atm. emission')
axs[1].plot(Net_flux3,atm_ck.play, label='w rayleigh')
axs[2].plot(atm_ck.tlay,atm_ck.play)
axs[0].set_xlabel('Heating Rate (W/kg)')
axs[1].set_xlabel('Flux (W/m^2)')
axs[2].set_xlabel('Temperature (K)')
axs[0].set_ylabel('Pressure (Pa)')
axs[0].invert_yaxis()
axs[0].legend()
axs[1].legend()
#axs[0].set_xlim(left=1.e-8)
fig.tight_layout()

# Modeling atmospheric evolution with `Atm_evolution`